In [2]:
import os
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
RAW_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
NEW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/offspring/12/snpDir/"
NEW_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/offspring/12/Result/"
REF_PATH = "/data1/zhoujb/cowpeaSequence/refernce/"
TEMP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/templateSNP/"

In [3]:
def getIdentity(dir_name):
    
    tmep_dir = os.path.join(TEMP_PATH, dir_name)
    tmep_df = pd.DataFrame()
    for tmep_file in os.listdir(tmep_dir):
        tmp_file = pd.read_table(os.path.join(tmep_dir, tmep_file), index_col="Index")
        tmep_df = pd.concat([tmep_df, tmp_file], axis=1, sort=False)

    four_raw = pd.read_table(os.path.join(NEW_SNP_PATH, "{}.txt".format(dir_name)), index_col="Pos")
    four_raw = four_raw.drop(columns=['Chr'])

    if len(four_raw) != len(tmep_df):
        raise ValueError("{}".format(dir_name))

    tmep_df = tmep_df.loc[four_raw.index]
    out_df = pd.DataFrame(columns=four_raw.columns, index=tmep_df.columns)
    for col in out_df.columns:
        for index in out_df.index:
            comb_df = pd.concat([tmep_df[index], four_raw[col]], axis=1, sort=False)
            identity = (comb_df[index] == comb_df[col]).sum() / comb_df.shape[0]
            out_df.loc[index, col] = identity

    out_df.to_csv(os.path.join(NEW_SNP_PATH, "{}.identity".format(dir_name)), sep="\t")
    return

if __name__ == "__main__":
    temp_dir = os.listdir(TEMP_PATH)

    with WorkerPool(n_jobs=48) as pool:
        pool.map(getIdentity, temp_dir, progress_bar=True)

    print("ALL DONE!")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 805/805 [00:25<00:00,  4.93it/s]


ALL DONE!


In [4]:
raw_gen_df = pd.read_table(os.path.join(RAW_RES_PATH, "Genotype_by_Hap_ID.tsv"), 
                           usecols=['Region_index', 'Chr', 'Start', 'End'], index_col='Region_index')
geno_hap_four = raw_gen_df.copy()
for index in raw_gen_df.index:
    tmp_file = pd.read_table(os.path.join(NEW_SNP_PATH, "{}.identity".format(index)), index_col=0)
    tmp_hap = tmp_file.idxmax(axis=0)
    
    for tmp_index in tmp_hap.index:
        if tmp_file[tmp_index].max() < 0.5:
            tmp_hap.loc[tmp_index] = "-"
    geno_hap_four.loc[index, tmp_hap.index] = tmp_hap.values

geno_hap_four.to_csv(os.path.join(NEW_RES_PATH, "Genotype_by_Hap_four.tsv"), sep="\t")
print("ALL DONE!")

ALL DONE!
